In [10]:
import torch
from torch import nn 
from torch.nn import functional as F 

net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X=torch.rand(2,20)
net(X)

tensor([[-0.3179,  0.0007, -0.0335,  0.0871,  0.1131, -0.0753, -0.0247, -0.1467,
         -0.2604, -0.0639],
        [-0.2979, -0.1670,  0.0241,  0.0413,  0.0822, -0.1665, -0.0021, -0.0516,
         -0.1753, -0.0635]], grad_fn=<AddmmBackward0>)

In [11]:
class MLP(nn.Module):# 任何一个层都是模组的子类
    """多层感知机，256个隐藏单元和一个10维输出层"""
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)# 隐藏层
        self.out=nn.Linear(256,10)# 输出层
# 定义模型的前向传播，根据输入x返回所需的模型输出
    def forward(self, X):    # 把输入放到hidden层
        return self.out(F.relu(self.hidden(X)))


In [12]:
net=MLP()
net(X)

tensor([[ 0.0594,  0.0575, -0.0780, -0.1440, -0.1252,  0.1396,  0.1953,  0.1378,
          0.0134,  0.1048],
        [ 0.0475,  0.0423, -0.1035, -0.1871, -0.0798,  0.1653,  0.1742, -0.0463,
         -0.0665,  0.0264]], grad_fn=<AddmmBackward0>)

In [13]:
#Sequential的设计是为了把其他模块串起来
# 我们需要定义两个关键函数：
# 一个将块逐个追加到列表中的函数
# 一种前向传播函数  用于把输入按追加块的顺序传递给块组成的链条


class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [14]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[ 0.1304, -0.1117,  0.1869, -0.1224,  0.0128,  0.0457, -0.0824,  0.0260,
          0.0114,  0.0506],
        [ 0.0249, -0.1754,  0.1741, -0.1035,  0.0630, -0.0044, -0.0850,  0.0670,
         -0.0902, -0.0030]], grad_fn=<AddmmBackward0>)

In [15]:
# Sequential类可以让我们组合新的架构  但是不是所有架构都是简单的顺序架构  当需要更强的灵活性时，我们需要定义自己的块

# 有时我没希望合并既不是上一层的结果也不是可更新参数的项
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数  在训练期间保持不变 不会参与训练
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X=self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        # 服用全连接层 相当于两个全连接层共享参数
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()


In [16]:
net = FixedHiddenMLP()
net(X)

tensor(0.0073, grad_fn=<SumBackward0>)

In [17]:
class NestMLP(nn.Module):#都是子类
    def __init__(self):# 把层定义好
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))

chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)


tensor(-0.2354, grad_fn=<SumBackward0>)